# Load conf files

## Input API

In [4]:
import tomli
import re
from uuid import uuid4

db_inputs_conf = 'examples/db_inputs.conf'
db_connections_conf = 'examples/db_connections.conf'

with open(db_connections_conf) as f:
    tmp = f.read()
    tmp_formatted = re.sub(r"(\s*=\s*)(.*)", r"\1'\2'", tmp)
    tomli_db_connections = tomli.loads(tmp_formatted).items()
tomli_db_connections

In [21]:
with open(db_inputs_conf) as f:
    tmp = f.read()
    tmp = re.sub(r"\\\n"," ",tmp)
    tmp_formatted = re.sub(r"(\s*=\s*)(.*)", r"\1'''\2'''", tmp)
    tomli_db_inputs = tomli.loads(tmp_formatted).items()
tomli_db_inputs

dict_items([('assignmentgroups', {'connection': 'TOOLX_P', 'disabled': '0', 'host': 'host1', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 6 * * *', 'max_rows': '100000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"', 'source': 'toolx', 'sourcetype': 'assignmentgroups'}), ('CI_relations', {'connection': 'TOOLX_P', 'disabled': '0', 'host': 'host1', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 5 * * *', 'max_rows': '1000000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"', 'source': 'toolx', 'sourcetype': 'ci_relations'}), ('config_full', {'connection': 'TOOLY_P', 'description': 'Get all config daily', 'disabled': '0', 'host': 'host2', 'index': 'indexy', 'index_time_mode': 'current', 'interval': '0 4 * * *', 'max_rows': '1000000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."CONFIG_SPLUNK"', 'query_timeout': '300', 'source': 'tooly', 'sourcetype': 'tooly'}), ('input_example01', {'connection'

In [29]:
with open(db_connections_conf) as f:
    tmp = f.read()
    tmp = re.sub(r"\\\n"," ",tmp)
    tmp_formatted = re.sub(r"(\s*=\s*)(.*)", r"\1'''\2'''", tmp)
    tomli_db_ = tomli.loads(tmp_formatted).items()
tomli_db_

dict_items([('TOOLX_P', {'connection_type': 'oracle_service', 'customizedJdbcUrl': 'jdbc:oracle:thin:@(DESCRIPTION_LIST=(LOAD_BALANCE=off)(FAILOVER=on)(DESCRIPTION=(CONNECT_TIMEOUT=5)(TRANSPORT_CONNECT_TIMEOUT=3)(RETRY_COUNT=3)(ADDRESS_LIST=(LOAD_BALANCE=on)(ADDRESS=(PROTOCOL=TCP)(HOST=host.sample.com)(PORT=39000)))(CONNECT_DATA=(SERVICE_NAME=DATABASESAMPLENAME)))(DESCRIPTION=(CONNECT_TIMEOUT=5)(TRANSPORT_CONNECT_TIMEOUT=3)(RETRY_COUNT=3)(ADDRESS_LIST=(LOAD_BALANCE=on)(ADDRESS=(PROTOCOL=TCP)(HOST=host.sample.com)(PORT=39000)))(CONNECT_DATA=(SERVICE_NAME=DATABASESAMPLENAME))))', 'database': 'DATABASESAMPLENAME', 'disabled': '0', 'host': 'host.sample.com', 'identity': 'TOOLX_P', 'jdbcUseSSL': 'false', 'localTimezoneConversionEnabled': 'false', 'port': '39000', 'readonly': 'true', 'timezone': 'Europe/Amsterdam'}), ('TOOLY_P', {'connection_type': 'oracle_service', 'database': 'DATABASESAMPLENAMEY', 'disabled': '0', 'host': 'host2.cinqict.nl', 'identity': 'TOOLY_P', 'jdbcUseSSL': 'false', '

In [5]:
#split string in new lines
with open(db_inputs_conf) as f:
    tmp = f.read()
    transformed_lines = []
    multiline_query = False
    for line in tmp.split('\n'):
        line = re.sub(r"\s*=\s*", " = ", line)
        if 'query' in line:
            if '\\' in line:
                query_content = line.split('query = ')[1].rstrip("\\").strip()
                # Append the modified 'query' line with triple quotes
                transformed_lines.append("query = '''")
                # Append the original content on a new line
                transformed_lines.append(query_content)
                multiline_query = True
            else:
                # Add other lines as they are
                transformed_lines.append(line)
                multiline_query = False

        elif multiline_query == True and '\\' in line:
            transformed_lines.append(line)
            multiline_query = True

        elif multiline_query == True and '\\' not in line:
            transformed_lines.append(line)
            transformed_lines.append("'''")
            multiline_query = False
        else:
            transformed_lines.append(line)

transformed_text = "\n".join(transformed_lines)
text = transformed_text.strip()
# Wrap the text with triple double quotes, starting and ending on a new line
formatted_text = f"""\n{text}\n"""
tmp = re.sub(r"\\\n", "\n", formatted_text)
tmp = re.sub(r"(\s*=\s*)(.*)", r"\1'\2'", tmp)
tomli_db_inputs = tomli.loads(tmp).items()
tomli_db_inputs

dict_items([('assignmentgroups', {'connection': 'TOOLX_P', 'disabled': '0', 'host': 'host1', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 6 * * *', 'max_rows': '100000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"', 'source': 'toolx', 'sourcetype': 'assignmentgroups'}), ('CI_relations', {'connection': 'TOOLX_P', 'disabled': '0', 'host': 'host1', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 5 * * *', 'max_rows': '1000000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"', 'source': 'toolx', 'sourcetype': 'ci_relations'}), ('config_full', {'connection': 'TOOLY_P', 'description': 'Get all config daily', 'disabled': '0', 'host': 'host2', 'index': 'indexy', 'index_time_mode': 'current', 'interval': '0 4 * * *', 'max_rows': '1000000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."CONFIG_SPLUNK"', 'query_timeout': '300', 'source': 'tooly', 'sourcetype': 'tooly'}), ('input_example01', {'connection'

# Re organize data to make 2 batches, one for the Input API and another for Connections API

## Input API

In [22]:
merged_data = {}
dict_db_inputs = dict(tomli_db_inputs)
dict_db_connections = dict(tomli_db_connections)

for key, input_value in dict_db_inputs.items():
    connection_key = input_value.get("connection")
    connection_data = dict_db_connections.get(connection_key, {})
    merged_data[key] = {**input_value, **connection_data}

# Convert values

## schedule.enabled -> disabled

Because Splunk talks about disabled and Cribl about enabled the states need to be transformed to the opposite value. Cribl only allows true or false booleans where Splunk allows 0 and 1 numbers as well; these need to be transformed.

| From | To |
| :---- | :---- |
| true | false |
| false | true |
| 1 | false |
| 0 | true |


In [23]:
def transform_disabled_values(data):
    for key, sub_dict in data.items():
        if 'disabled' in sub_dict:
            value = sub_dict['disabled']
            if isinstance(value, str):
                value = value.lower()
            if value in ['true', 1, True, '1']:
                sub_dict['disabled'] = False
            elif value in ['false', 0, False, '0']:
                sub_dict['disabled'] = True
    return data

transformed_merged_data = transform_disabled_values(merged_data)


In [24]:
tomli_input = transformed_merged_data.items()

In [25]:
tomli_input

dict_items([('assignmentgroups', {'connection': 'TOOLX_P', 'disabled': True, 'host': 'host.sample.com', 'index': 'indexx', 'index_time_mode': 'current', 'interval': '0 6 * * *', 'max_rows': '100000', 'mode': 'batch', 'query': 'SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"', 'source': 'toolx', 'sourcetype': 'assignmentgroups', 'connection_type': 'oracle_service', 'customizedJdbcUrl': 'jdbc:oracle:thin:@(DESCRIPTION_LIST=(LOAD_BALANCE=off)(FAILOVER=on)(DESCRIPTION=(CONNECT_TIMEOUT=5)(TRANSPORT_CONNECT_TIMEOUT=3)(RETRY_COUNT=3)(ADDRESS_LIST=(LOAD_BALANCE=on)(ADDRESS=(PROTOCOL=TCP)(HOST=host.sample.com)(PORT=39000)))(CONNECT_DATA=(SERVICE_NAME=DATABASESAMPLENAME)))(DESCRIPTION=(CONNECT_TIMEOUT=5)(TRANSPORT_CONNECT_TIMEOUT=3)(RETRY_COUNT=3)(ADDRESS_LIST=(LOAD_BALANCE=on)(ADDRESS=(PROTOCOL=TCP)(HOST=host.sample.com)(PORT=39000)))(CONNECT_DATA=(SERVICE_NAME=DATABASESAMPLENAME))))', 'database': 'DATABASESAMPLENAME', 'identity': 'TOOLX_P', 'jdbcUseSSL': 'false', 'localTimezoneConversionEnabl

# Schemas that include aliases

## Inputs API Schema

In [6]:
from pydantic import BaseModel, Field, AliasChoices
from typing import List, Optional
schema = BaseModel

get the right input schema according to curl. Include default values. Include aliases.

In [44]:
from pydantic import BaseModel, Field
from typing import List, Optional
from uuid import uuid4

class Metadata(BaseModel):
    name: str
    value: str

class Run(BaseModel):
    mode: str

class Schedule(BaseModel):
    cronSchedule: str = Field(alias='interval')
    run: Run
    enabled: bool = Field(alias='disabled')

class CollectorConf(BaseModel):
    connectionId: str = Field(alias='connection')
    query: str

class Collector(BaseModel):
    conf: CollectorConf
    type: str = "database"

class InputType(BaseModel):
    type: str = "collection"
    metadata: List[Metadata]

class InputSchema(BaseModel):
    type: Optional[str] = "collection"
    schedule: Schedule
    collector: Collector
    input: InputType
    id: Optional[str]

# Transform `tomli_input` data to match `metadata` requirements
def create_metadata(data):
    return [
        Metadata(name="host", value=data["host"]),
        Metadata(name="index", value=data["index"]),
        Metadata(name="source", value=data.get("source", "")),
        Metadata(name="sourcetype", value=data["sourcetype"])
    ]

# Parsing tomli_input to create InputSchema instances
result = [
    InputSchema(
        schedule=Schedule(
            interval=row['interval'],
            run=Run(mode=row['mode']),
            disabled=row['disabled']
        ),
        collector=Collector(
            conf=CollectorConf(
                connection=row['connection'],
                query=row['query']
            )
        ),
        input=InputType(
            metadata=create_metadata(row)
        ),
        id=f"{key}-{uuid4()}"
    )
    for key, row in tomli_input
]


In [27]:
result[0]

InputSchema(type='collection', schedule=Schedule(cronSchedule='0 6 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"'), type='database'), input=InputType(type='collection', metadata=[Metadata(name='host', value='host.sample.com'), Metadata(name='index', value='indexx'), Metadata(name='source', value='toolx'), Metadata(name='sourcetype', value='assignmentgroups')]), id='assignmentgroups-6fcf76c9-fb7f-4712-8af0-eaa4602ae78d')

In [28]:
def to_dict(obj):
    """Recursively converts an object to a dictionary."""
    if isinstance(obj, list):
        return [to_dict(item) for item in obj]
    elif hasattr(obj, "__dict__"):
        result = {key: to_dict(value) for key, value in vars(obj).items()}
        return result
    else:
        return obj

input_schema_list = [to_dict(i) for i in result]
for i, j in zip(input_schema_list, result):
    i['input']['metadata'] = [
    {'name': meta.name, 'value': meta.value} for meta in j.input.metadata
]
input_schema_list

[{'type': 'collection',
  'schedule': {'cronSchedule': '0 6 * * *',
   'run': {'mode': 'batch'},
   'enabled': True},
  'collector': {'conf': {'connectionId': 'TOOLX_P',
    'query': 'SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"'},
   'type': 'database'},
  'input': {'type': 'collection',
   'metadata': [{'name': 'host', 'value': 'host.sample.com'},
    {'name': 'index', 'value': 'indexx'},
    {'name': 'source', 'value': 'toolx'},
    {'name': 'sourcetype', 'value': 'assignmentgroups'}]},
  'id': 'assignmentgroups-6fcf76c9-fb7f-4712-8af0-eaa4602ae78d'},
 {'type': 'collection',
  'schedule': {'cronSchedule': '0 5 * * *',
   'run': {'mode': 'batch'},
   'enabled': True},
  'collector': {'conf': {'connectionId': 'TOOLX_P',
    'query': 'SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"'},
   'type': 'database'},
  'input': {'type': 'collection',
   'metadata': [{'name': 'host', 'value': 'host.sample.com'},
    {'name': 'index', 'value': 'indexx'},
    {'name': 'source', 'value': 'toolx'},


In [45]:
for i in result:
    i.input.metadata = [{'name': meta.name, 'value': meta.value} for meta in i.input.metadata]

result


[InputSchema(type='collection', schedule=Schedule(cronSchedule='0 6 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host.sample.com'}, {'name': 'index', 'value': 'indexx'}, {'name': 'source', 'value': 'toolx'}, {'name': 'sourcetype', 'value': 'assignmentgroups'}]), id='assignmentgroups-739e3e33-c6b9-4aa8-95e4-bafcca2c66d6'),
 InputSchema(type='collection', schedule=Schedule(cronSchedule='0 5 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host.sample.com'}, {'name': 'index', 'value': 'indexx'}, {'name': 'source', 'value': 'toolx'}, {'name': 'sourcetype', 'value': 'ci_relations'}

In [53]:
result[0].collector.conf.query

'SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"'

In [54]:
def transform_to_js_expression(query: str) -> str:
    """
    Transforms a SQL query string into a valid JavaScript expression
    for Cribl, enclosed in backticks.
    """
    # Escape double quotes for JavaScript compatibility
    escaped_query = query.replace('"', '\\"')
    # Wrap the query in backticks
    js_expression = f"`{escaped_query}`"
    return js_expression

for i in result:
    i.collector.conf.query = transform_to_js_expression(i.collector.conf.query)

print(result)


[InputSchema(type='collection', schedule=Schedule(cronSchedule='0 6 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='`SELECT * FROM \\"OWNER\\".\\"ASSIGNMENT_GROUPS_SPLUNK\\"`'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host.sample.com'}, {'name': 'index', 'value': 'indexx'}, {'name': 'source', 'value': 'toolx'}, {'name': 'sourcetype', 'value': 'assignmentgroups'}]), id='assignmentgroups-739e3e33-c6b9-4aa8-95e4-bafcca2c66d6'), InputSchema(type='collection', schedule=Schedule(cronSchedule='0 5 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='`SELECT * FROM \\"OWNER\\".\\"RELATIONSHIP_SPLUNK\\"`'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host.sample.com'}, {'name': 'index', 'value': 'indexx'}, {'name': 'source', 'value': 'toolx'}, {'name': 'sourcetype', 'valu

In [10]:
[InputSchema(type='collection', schedule=Schedule(cronSchedule='0 6 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='SELECT * FROM "OWNER"."ASSIGNMENT_GROUPS_SPLUNK"'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host.sample.com'}, {'name': 'index', 'value': 'indexx'}, {'name': 'source', 'value': 'toolx'}, {'name': 'sourcetype', 'value': 'assignmentgroups'}]), id='assignmentgroups-6e2e8df4-704b-4b79-a2b8-7203b99f0407'), InputSchema(type='collection', schedule=Schedule(cronSchedule='0 5 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLX_P', query='SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host.sample.com'}, {'name': 'index', 'value': 'indexx'}, {'name': 'source', 'value': 'toolx'}, {'name': 'sourcetype', 'value': 'ci_relations'}]), id='CI_relations-225e5433-1d2a-40a1-8b1c-83dd48f36051'), InputSchema(type='collection', schedule=Schedule(cronSchedule='0 4 * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='TOOLY_P', query='SELECT * FROM "OWNER"."CONFIG_SPLUNK"'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'host2.cinqict.nl'}, {'name': 'index', 'value': 'indexy'}, {'name': 'source', 'value': 'tooly'}, {'name': 'sourcetype', 'value': 'tooly'}]), id='config_full-399206e8-6513-4719-ba83-e7330bc3396c'), InputSchema(type='collection', schedule=Schedule(cronSchedule='*/5 * * * *', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='connA', query='select UID,SUBSTRING(field,1,5000) AS Field from Database;'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'database.cinqict.nl'}, {'name': 'index', 'value': 'index01'}, {'name': 'source', 'value': ''}, {'name': 'sourcetype', 'value': 'sourcetype01'}]), id='input_example01-2a91a25b-5617-4a40-9364-655bc4558f32'), InputSchema(type='collection', schedule=Schedule(cronSchedule='0 * * * 1-5', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='connD', query='SELECT count(DISTINCT USER) AS "USERS" FROM sys.v_$session S WHERE S.USERNAME NOT IN ( \'SYS\', \'SYSTEM\' ) AND S.USERNAME NOT LIKE USER  AND S.PROGRAM =\'someprogram.exe\' ORDER BY USER'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'database-windows.cinqict.nl'}, {'name': 'index', 'value': 'index02'}, {'name': 'source', 'value': 'source02'}, {'name': 'sourcetype', 'value': 'sourcetype02'}]), id='input_example02-62991461-438b-43d1-9ac1-ed4f1685d2a3'), InputSchema(type='collection', schedule=Schedule(cronSchedule='3600', run=Run(mode='batch'), enabled=True), collector=Collector(conf=CollectorConf(connectionId='connC', query='SELECT * FROM "db"."dbo"."FunctionalUser"'), type='database'), input=InputType(type='collection', metadata=[{'name': 'host', 'value': 'database-oracle.cinqict.nl'}, {'name': 'index', 'value': 'index03'}, {'name': 'source', 'value': 'source03'}, {'name': 'sourcetype', 'value': 'sourcetype03'}]), id='input_example03-5e85b0ec-e3f5-4528-839e-2a3e397fe968')]
[{'status': 'error', 'message': 'Failed to post new input. Response: {\'status\': \'error\', \'message\': \'[{"keyword":"jsExpression","message":"Unexpected string","dataPath":"/query","params":{"expression":"SELECT * FROM \\\\"OWNER\\\\".\\\\"ASSIGNMENT_GROUPS_SPLUNK\\\\""},"schemaPath":"#/properties/query/jsExpression"}]\'}'}, {'status': 'error', 'message': 'Failed to post new input. Response: {\'status\': \'error\', \'message\': \'[{"keyword":"jsExpression","message":"Unexpected string","dataPath":"/query","params":{"expression":"SELECT * FROM \\\\"OWNER\\\\".\\\\"RELATIONSHIP_SPLUNK\\\\""},"schemaPath":"#/properties/query/jsExpression"}]\'}'}, {'status': 'error', 'message': 'Failed to post new input. Response: {\'status\': \'error\', \'message\': \'[{"keyword":"jsExpression","message":"Unexpected string","dataPath":"/query","params":{"expression":"SELECT * FROM \\\\"OWNER\\\\".\\\\"CONFIG_SPLUNK\\\\""},"schemaPath":"#/properties/query/jsExpression"}]\'}'}, {'status': 'error', 'message': 'Failed to post new input. Response: {\'status\': \'error\', \'message\': \'[{"keyword":"jsExpression","message":"Unexpected identifier","dataPath":"/query","params":{"expression":"select UID,SUBSTRING(field,1,5000) AS Field from Database;"},"schemaPath":"#/properties/query/jsExpression"}]\'}'}, {'status': 'error', 'message': 'Failed to post new input. Response: {\'status\': \'error\', \'message\': \'[{"keyword":"jsExpression","message":"Unexpected identifier","dataPath":"/query","params":{"expression":"SELECT count(DISTINCT USER) AS \\\\"USERS\\\\" FROM sys.v_$session S WHERE S.USERNAME NOT IN ( \\\'SYS\\\', \\\'SYSTEM\\\' ) AND S.USERNAME NOT LIKE USER  AND S.PROGRAM =\\\'someprogram.exe\\\' ORDER BY USER"},"schemaPath":"#/properties/query/jsExpression"}]\'}'}, {'status': 'error', 'message': 'Failed to post new input. Response: {\'status\': \'error\', \'message\': \'[{"keyword":"jsExpression","message":"Unexpected string","dataPath":"/query","params":{"expression":"SELECT * FROM \\\\"db\\\\".\\\\"dbo\\\\".\\\\"FunctionalUser\\\\""},"schemaPath":"#/properties/query/jsExpression"}]\'}'}]


{'items': [{'type': 'collection',
   'schedule': {'cronSchedule': '0 6 * * *',
    'run': {'mode': 'batch'},
    'enabled': True},
   'collector': {'conf': {'connectionId': 'default_connection',
     'query': '"SELECT * FROM OWNER.CONFIG_SPLUNK"'},
    'type': 'database'},
   'input': {'type': 'collection',
    'metadata': [{'name': 'index', 'value': 'default_index'},
     {'name': 'sourcetype', 'value': 'default_sourcetype'},
     {'name': 'source', 'value': 'default_source'}]},
   'connection': 'TOOLX_P',
   'disabled': 0,
   'host': 'host1',
   'index': 'indexx',
   'index_time_mode': 'current',
   'interval': '0 5 * * *',
   'max_rows': 1000000,
   'mode': 'batch',
   'query': 'SELECT * FROM "OWNER"."RELATIONSHIP_SPLUNK"',
   'source': 'toolx',
   'sourcetype': 'ci_relations',
   'id': 'CI_relations-ecdb0b99-e8ec-4a60-9c03-b6042c84671d',
   'history': [],
   'savedState': {}}],
 'count': 1}

In [ ]:
class ConnectionSchema(Basemodel):
    id: str
    databaseType: str
    username: str
    password: str
    connectionString: str | None = None
    database: str
    disabled: int
    host: str
    identity: str
    jdbcUseSSL: bool
    localTimezoneConversionEnabled: bool
    port: int
    readonly: bool
    timezone: str | None = None
    

In [59]:
import re

def is_cron_format(value: str) -> bool:
    """
    Check if the given value is in a valid cron schedule format.

    Parameters
    ----------
    value : str
        The value to check.

    Returns
    -------
    bool
        True if the value is a valid cron schedule format, False otherwise.
    """
    if value.isdigit():
        return False

    # Cron schedule regex: Matches standard cron patterns
    cron_regex = r'^(\*|([0-5]?\d)) (\*|([01]?\d|2[0-3])) (\*|([0-2]?\d)) (\*|([1-7]))$|^@(?:yearly|annually|monthly|weekly|daily|hourly)$'
    
    return bool(re.match(cron_regex, value.strip()))

# Example Test Cases
intervals = [
    "0 6 * * *",  # Valid cron
    "*/5 * * * *",  # Valid cron
    "0 5 * * *",  # Valid cron
    "0 * * * 1-5",  # Valid cron
    "3600"  # Invalid (numeric interval)
]

# Test the function
for interval in intervals:
    print(f"Interval: {interval}, Is Cron Format: {is_cron_format(interval)}")


Interval: 0 6 * * *, Is Cron Format: False
Interval: */5 * * * *, Is Cron Format: False
Interval: 0 5 * * *, Is Cron Format: False
Interval: 0 * * * 1-5, Is Cron Format: False
Interval: 3600, Is Cron Format: False


In [58]:
is_cron_format('*/5 * * * *')

False

# Two load functions, one for the Iputs and one for Connections